# Assignment 1
## Digital Musicology

@authors: Joris Monnet, Xingyu Pan, Yutaka Osaki, Yiwei Liu

Due Date: 24/04/2024


In [1]:
import music21

## Task A

TODO: open and listen a midi file

- The speed varies near consecutive notes [ex. 0:00~1:00]
- When the sound is loud, the tempo is fast, and when the sound is soft, the tempo is slow[ ex. 1:10~]
- When the pitch is high, the tempo is fast [ex. 1:23]
- The tempo gradually quickens as the staccato continues [ex. 1:40~2:00]
- When moving from high to low notes, the tempo slows significantly.
- When the same recurrent phrase continues, the tempo becomes slow [ex. 2:40~3:15]
- When it suddenly becomes higher pitched, the tempo slows down at the end [ex. 4:10~]
- The tempo is more likely to change when a few notes are played at once. [ex. 10:00~]

# Task B

From your analysis in task A, think about how to model these aspects computationally. In this task, choose one or more aspect(s) in your analysis you would like to model.

You should write an algorithm that outputs a MIDI file from an musicXML or the unperformed MIDI. In your report, please clearly describe your model.

### Import original midi

In [3]:
path_to_file = "../Mephisto_Waltz/"

In [4]:
original_midi_path = f"{path_to_file}midi_score.mid"
oritinal_txt_path  = f"{path_to_file}midi_score.txt"
original_midi = music21.converter.parse(original_midi_path)

### Functions

In [5]:
def generate_expressive_MIDI(original_midi):
    midifile = original_midi
    return midifile

### Get Pitchs and Volume

In [60]:
def get_pitches(score):
    print("----------- Start Getting Pitches -----------")
    pitches = []
    for element in score.recurse():
        if isinstance(element, music21.note.Note):
            pitches.append(element.pitch.frequency)
        elif isinstance(element, music21.chord.Chord):
            pitches.extend([n.pitch.frequency for n in element.notes])
    print("----------- Finish Getting Pitches -----------")
    return sorted(set(pitches))

def get_volume(score):
    print("----------- Start Getting Volume -----------")
    velocities = [note.volume.velocity for note in score.recurse().notes]
    unique_velocities = list(set(velocities))
    unique_velocities.sort()
    print("----------- Finish Getting Volume -----------")
    return unique_velocities

### Edit Tempo and Length

In [61]:
def edit_tempo_by_pitches(score, pitches_sorted, factor=2.00):
    print("----------- Start Editing Tempo -----------")
    high_pitches, low_pitches = pitches_sorted[len(pitches_sorted) // 2:], pitches_sorted[:len(pitches_sorted) // 2]
    for element in score.recurse():
        if isinstance(element, music21.note.Note) or isinstance(element, music21.chord.Chord):
            element_pitches = [element.pitch] if isinstance(element, music21.note.Note) else [n.pitch for n in element.notes]
            current_tempo = element.getContextByClass(music21.tempo.MetronomeMark)
            if current_tempo:
                for pitch in element_pitches:
                    if pitch in high_pitches:
                        index = high_pitches.index(pitch)
                        new_tempo = music21.tempo.MetronomeMark(number=current_tempo.number * (factor ** ((index+1) / len(high_pitches))))
                        score.insert(element.offset, new_tempo)
                        break
                    elif pitch in low_pitches:
                        index = low_pitches.index(pitch)
                        new_tempo = music21.tempo.MetronomeMark(number=current_tempo.number / (factor ** ((index+1) / len(low_pitches))))
                        score.insert(element.offset, new_tempo)
                        break
    print("----------- Finish Editing Tempo -----------")
    return score
def edit_NoteLength_by_volume(score, unique_velocities, factor=2.00):
    print("----------- Start Editing Note Length -----------")
    high_velocities, low_velocities = unique_velocities[len(unique_velocities) // 2:], unique_velocities[:len(unique_velocities) // 2]
    for note in score.recurse().notes:
        if note.volume.velocity in high_velocities:
            index = high_velocities.index(note.volume.velocity)
            note.quarterLength *= factor ** -(index / len(high_velocities))
        elif note.volume.velocity in low_velocities:
            index = low_velocities.index(note.volume.velocity)
            note.quarterLength *= factor ** ((len(low_velocities) - 1 - index) / len(low_velocities))

    print("----------- Finish Editing Note Length -----------")
    return score

### Execute functions

In [62]:
def adjust_midi_tempo(midi_file, factor = 2.00):
    try:
        score = music21.converter.parse(midi_file)
    except:
        score = midi_file
    pitches_sorted = get_pitches(score)
    edited_score = edit_tempo_by_pitches(score, pitches_sorted, factor)
    return edited_score
def adjust_midi_velocity(midi_file, factor = 2.00):
    try:
        score = music21.converter.parse(midi_file)
    except:
        score = midi_file
    unique_velocities = get_volume(score)
    score = edit_NoteLength_by_volume(score, unique_velocities, factor)
    return score

### Fucntion to Save

In [66]:
import os
def save_midi(score, directory="edited", base_filename="edited"):
    print("----------- Start Downloading midi file -----------")
    if not os.path.exists(directory):
        os.makedirs(directory)
    files = os.listdir(directory)
    count = sum(1 for file in files if file.startswith(base_filename) and file.endswith(".midi"))
    new_filename = f"{base_filename}_{count + 1:02d}.midi"
    file_path = os.path.join(directory, new_filename)
    mf = music21.midi.translate.music21ObjectToMidiFile(score)
    mf.open(file_path, 'wb')
    mf.write()
    mf.close()
    print("----------- Finish Downloading midi file -----------")
    return file_path

### Execution

In [65]:
edited_score_by_tempo = adjust_midi_tempo(original_midi_path)
edited_score_by_volume = adjust_midi_velocity(edited_score_by_tempo)
save_midi(edited_score_by_volume)

----------- Start Getting Pitches -----------
----------- Finish Getting Pitches -----------
----------- Start Editing Tempo -----------
----------- Finish Editing Tempo -----------
----------- Start Getting Volume -----------
----------- Finish Getting Volume -----------
----------- Start Editing Note Length -----------
----------- Finish Editing Note Length -----------
----------- Start Downloading midi file -----------
----------- Finish Downloading midi file -----------


'edited/edited_08.midi'

## Task C

Compare your generated MIDI with both the unperformed MIDI and the human performace recording. Discuss their differences and potentially what and how you can further improve. This should be part of your discussion section in the report.

In [ ]:
# TODO visualizations of differences ?